https://adventofcode.com/2016/day/11

https://www.mathsisfun.com/chicken_crossing.html

In [36]:
from itertools import combinations
from collections import deque
from toolz import concat

fs = frozenset

In [69]:
elements = ['Sr', 'Pu', 'Tm', 'Ru', 'Cm']

In [59]:
def is_legal(floor, elements=elements):
    for e in elements:
        if e+'M' in floor and e+'G' not in floor and get_gens(floor):
            return False
    return True

def get_gens(floor):
    return [x for x in floor if x.endswith('G')]

In [4]:
initial_state = (
    fs(),
    fs(['TmM']),
    fs(['TmG', 'RuG', 'RuM', 'CmG', 'CmM']),
    fs(['E', 'SrG', 'SrM', 'PuG', 'PuM']),
)
initial_state

(frozenset(),
 frozenset({'TmM'}),
 frozenset({'CmG', 'CmM', 'RuG', 'RuM', 'TmG'}),
 frozenset({'E', 'PuG', 'PuM', 'SrG', 'SrM'}))

In [5]:
[is_legal(floor) for floor in initial_state]

[True, True, True, True]

In [6]:
all(is_legal(floor) for floor in initial_state)

True

In [63]:
def iter_next_state(state, elements=elements):
    floors = [(i, set(s)) for (i, s) in enumerate(state)]
    source_index, source = next((i, f) for (i, f) in floors if 'E' in f)
    targets = [(i, f) for (i, f) in floors if i != source_index]
    source.remove('E')
    
    for combo in concat([combinations(source, 2), combinations(source, 1)]):
        sc = source.copy()
        for item in combo:
            sc.discard(item)
        if is_legal(sc, elements):
            for (target_index, target) in targets:
                if abs(target_index - source_index) > 1:
                    continue
                tc = target.copy()
                for item in combo:
                    tc.add(item)
                tc.add('E')
                if is_legal(tc, elements):
                    L = sorted([(source_index, sc), (target_index, tc)]
                                + [(i, f) for (i, f) in targets if i != target_index])
                    yield tuple(fs(y) for (x, y) in L)

In [64]:
it = iter_next_state(initial_state)

In [65]:
next(it)

(frozenset(),
 frozenset({'TmM'}),
 frozenset({'CmG', 'CmM', 'E', 'PuG', 'RuG', 'RuM', 'SrG', 'TmG'}),
 frozenset({'PuM', 'SrM'}))

In [66]:
is_legal(frozenset({'PuM', 'SrG', 'SrM'}))

False

In [67]:
def bfs(initial_state, goal_test=None, elements=elements):
    if goal_test is None:
        goal_test = lambda state: not any(state[1:])
    frontier = deque([(initial_state, 0)]) # deque of (state, steps)
    visited = set()
    while frontier:
        state, count = frontier.popleft()
        if goal_test(state):
            print('All done: ', count)
            return count
        if state in visited:
            continue
        visited.add(state)
        count += 1
        for next_state in iter_next_state(state, elements=elements):
            if next_state not in visited:
                frontier.append((next_state, count))


In [68]:
%%time
initial_state = (
    fs(),
    fs(['TmM']),
    fs(['TmG', 'RuG', 'RuM', 'CmG', 'CmM']),
    fs(['E', 'SrG', 'SrM', 'PuG', 'PuM']),
)
out = bfs(initial_state)

All done:  37
CPU times: user 6.72 s, sys: 55.8 ms, total: 6.77 s
Wall time: 6.78 s


## part 2

In [70]:
elements = ['Sr', 'Pu', 'Tm', 'Ru', 'Cm', 'El', 'Di']

In [71]:
%%time
initial_state = (
    fs(),
    fs(['TmM']),
    fs(['TmG', 'RuG', 'RuM', 'CmG', 'CmM']),
    fs(['E', 'SrG', 'SrM', 'PuG', 'PuM', 'ElG', 'ElM', 'DiG', 'DiM']),
)
out = bfs(initial_state, elements=elements)

All done:  61
CPU times: user 9min 26s, sys: 3.64 s, total: 9min 30s
Wall time: 9min 30s
